In [ ]:
from datetime import date

import stats
import sicris
import doctoral_mentors

In [ ]:
date_prefix = date.today().isoformat()
print('Archiving new doctoral_mentors data with timestamp prefix:', date_prefix)

Requirements for mentors at UL:

* Article 32, [doctoral studies rules](https://www.uni-lj.si/mma/prdr21/2021033113243049/):

> Mentorstvo
> 
> 32. člen
>  
> Mentor in somentor na doktorskem študiju je visokošolski učitelj UL z nazivom docent, izredni ali redni profesor oziroma znanstveni delavec z nazivom znanstveni sodelavec, višji znanstveni sodelavec ali znanstveni svetnik in ima izkazano raziskovalno aktivnost z ustrezno bibliografijo s širšega področja teme doktorske disertacije. Minimalni pogoj za izkazovanje raziskovalne aktivnosti mentorja in somentorja je določen s sklepom Senata UL, ki je objavljen na spletni strani Doktorske šole UL.

* Sklep Senata UL o minimalnih pogojih za izkazovanje raziskovalne aktivnosti, [rule](https://www.uni-lj.si/mma/sklep_pogoji_19/2019013012384769/):

> ... zadnjih petih letih doseže 150 Z točk po SICRIS in <b>hkrati doseže več kot 0 točk pri kazalcu pomembnih dosežkov A1/2.


In [ ]:
researchers = sicris.get()
mentors = doctoral_mentors.get()

In [ ]:
for rid, v in researchers.items():
    firstname = v['FNAME']
    lastname = v['LNAME']
    titles = set([d.get('RSRTTL', '').strip() for d in v['EMPLOY']])
    titles_codes = set([d.get('RSRTTLCODE', '').strip() for d in v['EMPLOY']])
    if 'RECAPITUALITION' not in v:
        Z = 0.0
        A12 = 0.0
    else:
        Z = float(v['RECAPITUALITION'][0]['Z'].replace(',', '.'))
        A12 = float(v['RECAPITUALITION'][0]['A12'].replace(',', '.'))
    
    if Z >= 150 and A12 > 0:
        eligible = len(titles_codes & set(['34', '35', '36', '4'])) > 0
        if mentors.get(rid, {}).get('manual_eligibility_override', False) == False and not eligible:
            print('need to check manually:', eligible, rid, firstname, lastname, Z, A12, titles, titles_codes)
    else:
        eligible = False       

    rec = {
        'firstname': firstname,
        'lastname': lastname,
        'titles': ", ".join(sorted(titles)),
        'Z': Z,
        'A12': A12,
        'eligible': eligible
    }

    if rid in mentors:
        for k, v in rec.items():  
            mentors[rid][k] = v  # update
    else:
        mentors[rid] = rec

In [ ]:
doctoral_mentors.update(mentors)

In [ ]:
eligible_mentors = len([m for m in mentors.values() if m['eligible'] or m.get('manual_eligibility_override', False)])
print('Eligible doctoral mentors:', eligible_mentors)

In [ ]:
stats.update('doctoral_mentors', eligible_mentors, date_prefix)

In [ ]:
print('DONE')